Adaptado por Bruno Menezes (https://github.com/brunoleomenezes) e será utilizado apenas para fins acadêmicos.

In [1]:
import torch
from torch import nn
from sklearn import datasets

In [2]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

print(device)

cuda


Previsão da progressão da diabetes.


In [3]:
diab = datasets.load_diabetes()
dados = diab.data
alvo = diab.target

print(dados.shape, alvo.shape)

print(dados[13])
print(alvo[13])
#442 amostrar com 10 caracteristicas (idade, sexo e etc) e 442 rótulos

(442, 10) (442,)
[ 0.00538306  0.05068012 -0.00189471  0.00810087 -0.00432087 -0.01571871
 -0.00290283 -0.00259226  0.03839325 -0.01350402]
185.0


Implementando o MLP

In [4]:
class diabClassifier(nn.Module):

  def __init__(self, input_size, hidden_size, out_size):
    super(diabClassifier, self).__init__()

    # Arquitetura da rede
    self.hidden  = nn.Linear(input_size, hidden_size)
    self.relu    = nn.ReLU()
    self.out     = nn.Linear(hidden_size, out_size)
    self.softmax = nn.Softmax(dim=-1)

  def forward(self, X):
    
    aspecto = self.relu(self.hidden(X))
    saida  = self.softmax(self.out(aspecto))

    return saida

input_size  = dados.shape[1]
hidden_size = 32
out_size    = 1  # Progressão da doença

net = diabClassifier(input_size, hidden_size, out_size).to(device) #Utilizando na GPU 

*MSELoss* https://pytorch.org/docs/stable/nn.html#mseloss

In [5]:
criterio = nn.MSELoss().to(device) #Rótulo e predição precisam ter a mesma dimensionalidade

# Utilizando GPU
Xtensor = torch.from_numpy(dados).float().to(device) # Convertendo para float
Ytensor = torch.from_numpy(alvo).float().to(device)

print(Xtensor.shape, Ytensor.shape)

torch.Size([442, 10]) torch.Size([442])


In [6]:
Xtensor

tensor([[ 0.0381,  0.0507,  0.0617,  ..., -0.0026,  0.0199, -0.0176],
        [-0.0019, -0.0446, -0.0515,  ..., -0.0395, -0.0683, -0.0922],
        [ 0.0853,  0.0507,  0.0445,  ..., -0.0026,  0.0029, -0.0259],
        ...,
        [ 0.0417,  0.0507, -0.0159,  ..., -0.0111, -0.0469,  0.0155],
        [-0.0455, -0.0446,  0.0391,  ...,  0.0266,  0.0445, -0.0259],
        [-0.0455, -0.0446, -0.0730,  ..., -0.0395, -0.0042,  0.0031]],
       device='cuda:0')

In [7]:
Ytensor

tensor([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,  69.,
        179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,  68., 245.,
        184., 202., 137.,  85., 131., 283., 129.,  59., 341.,  87.,  65., 102.,
        265., 276., 252.,  90., 100.,  55.,  61.,  92., 259.,  53., 190., 142.,
         75., 142., 155., 225.,  59., 104., 182., 128.,  52.,  37., 170., 170.,
         61., 144.,  52., 128.,  71., 163., 150.,  97., 160., 178.,  48., 270.,
        202., 111.,  85.,  42., 170., 200., 252., 113., 143.,  51.,  52., 210.,
         65., 141.,  55., 134.,  42., 111.,  98., 164.,  48.,  96.,  90., 162.,
        150., 279.,  92.,  83., 128., 102., 302., 198.,  95.,  53., 134., 144.,
        232.,  81., 104.,  59., 246., 297., 258., 229., 275., 281., 179., 200.,
        200., 173., 180.,  84., 121., 161.,  99., 109., 115., 268., 274., 158.,
        107.,  83., 103., 272.,  85., 280., 336., 281., 118., 317., 235.,  60.,
        174., 259., 178., 128.,  96., 12

In [8]:
predicao = net(Xtensor)

loss = criterio(predicao.squeeze(), Ytensor) #squeeze retira dimensões a mais
print(loss.data) # Média da distância quadradica entre a predição e dado real

tensor(28771.2168, device='cuda:0')


Algumas funções de perda no PyTorch: https://pytorch.org/docs/stable/nn.html#loss-functions